### Importing packages

In [4]:
import re, os
import nltk
from nltk.util import pad_sequence
from nltk.util import bigrams
from nltk.util import ngrams
from nltk.util import everygrams
from nltk.lm.preprocessing import pad_both_ends
from nltk.lm.preprocessing import flatten
from nltk.corpus import brown, reuters

### Loading APPLING1DAT.643 as corpus

In [2]:
data_path = "data/0643/APPLING1DAT.643"
with open(data_path) as fp:
    data = fp.read()

In [3]:
data[:80]

'$Punjabi\nstrang  strange  I felt very *\nbrake  break  at * time\nbrack  break  wh'

In [5]:
def load_data(data_path="data/0643/APPLING1DAT.643"):
    spell_corpus = []
    inp = []
    crct_ans = []
    with open(data_path) as fp:
        data = fp.read()
    for line in data.split('\n'):
        w = line.split()
        if len(w)>2:
            crct_ans.append(w[1])
            inp_ = " ".join(w[2:]).strip()
            inp.append(inp_)
            spell_corpus.append(re.sub(r"\*", w[1], inp_))
    return spell_corpus, inp, crct_ans
            
        
    

In [6]:
spell_corpus, inp, crct_ans = load_data()

In [7]:
spell_corpus[:5]

['I felt very strange',
 'at break time',
 'when the break was finished',
 'in the winter when it was snowing',
 'I thought it was a ghost']

In [8]:
def check_data_qaulity(corpus, word_interest):
    corpora = {'brown': brown, 'reuters': reuters}
    corp = corpora.get(corpus, brown)
    print(f'Inspecting {corpus} corpus')
    
    combined = {k:0 for k in word_interest}
    for cat in corp.categories():

        cat_words = corp.words(categories=cat)
        fdist = nltk.FreqDist(w.lower() for w in cat_words)
        t = 0
        for m in word_interest:
            print(m + ':', fdist[m], end=' ')
            t+=fdist[m]
            combined[m] += fdist[m]
        print("\n")
        print(f'{cat} -> {t}')
        print("="*40)
    print(combined)

### Selecting Primary Corpus for LM

In [10]:
# check whether the correct spelling from our corpus E is there in the train data or not
# dropped the plan as there was not much difference in result and difficult to report in single page format
# check_data_qaulity("brown", crct_ans)

In [34]:
data_prepfn = lambda all_data: [[w.lower() if w.isalpha() else w for w in sent if w.isalpha() or w.strip()=='.' or w.strip()==','] for sent in all_data]

### Final Experiment Wrapper

In [33]:
data = brown.sents(categories="news") # Corpus C on which LM will be trained
avgk, len_tseed = experiment("brown-newsK", data, "brown-news") 
# arg1->folder_name(save and load models from);arg2-> dataset;arg3->dataname for debugging
# also have implementation for bi-direction n-gram but as product of 2 LMs and is not giving good result.

Running experiment brown-newsK with data brown-news
Loading saved model on disk
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.

### Execute below codes to play with the code in above cell

In [35]:
def dump_modelK(out_, data_prep):
#     out_ = "ngrams_Brown"
    if not os.path.exists(out_): os.mkdir(out_)
    n_model = []
    n_l = [1, 2, 3, 5, 10]
    for n in n_l:
        train_data, padded_sents = padded_everygram_pipeline(n, data_prep)
        model = KneserNeyInterpolated(n) 
#         print(f'{n}-gram: With {model.vocab} vocab')
        model.fit(train_data, padded_sents)
        print(f'Training Complete for {n}-gram {model.vocab}')
    #     t_se = "<s> my friends, joan and arthur live near the".lower()
    #     generate_topk(model, k=10, text_seed=word_tokenize(t_se))
        n_model.append(model)
        with open(f'{out_}/{n}-grams.pkl', 'wb') as fout:
            pickle.dump(model, fout)
    return n_model

In [36]:
def dump_model(out_, data_prep):
#     out_ = "ngrams_Brown"
    if not os.path.exists(out_): os.mkdir(out_)
    n_model = []
    n_l = [1, 2, 3, 5, 10]
    for n in n_l:
        train_data, padded_sents = padded_everygram_pipeline(n, data_prep)
        model = MLE(n) 
#         print(f'{n}-gram: With {model.vocab} vocab')
        model.fit(train_data, padded_sents)
        print(f'Training Complete for {n}-gram {model.vocab}')
    #     t_se = "<s> my friends, joan and arthur live near the".lower()
    #     generate_topk(model, k=10, text_seed=word_tokenize(t_se))
        n_model.append(model)
        with open(f'{out_}/{n}-grams.pkl', 'wb') as fout:
            pickle.dump(model, fout)
    return n_model

In [37]:
import pickle

In [38]:
def evaluate_spell(inp, crct_ans, n_model):    
    
    if type(n_model)==str:
        models = []
        fnm = [f'{i}-grams.pkl' for i in [5, 10]]
        for f in fnm:
            with open(f"{n_model}/{f}", "rb") as fp:
                models.append(pickle.load(fp))
        n_model = models
        
    len_tseed = []
    
    score = {k.order:[0, 0, 0] for k in n_model}
        
    for i, (in_, ans_) in enumerate(zip(inp, crct_ans)):
        sear = re.search(r"\*", in_)
        if sear:
            ix = sear.start()
        else:
            ix = len(in_)
            print("Fracko")
        t_se = in_[:ix].lower()
        len_tseed.append(len(t_se.split()))
        
        for model in n_model:
            topk = generate_topk(model, k=10, text_seed=word_tokenize(t_se))
            avgk = report_successk(ans_, topk)
            print(avgk)
            for idx, succ_k in enumerate(avgk):
                score[model.order][idx]+=succ_k 
            
            if any(avgk):
                print(in_, ans_)
                print(model.order)
                print(t_se)
                print(topk)
                print(avgk)
    return {k:[elm/len(inp) for elm in v] for k, v in score.items()}, len_tseed

In [39]:
def evaluate_spell_bi(inp, crct_ans, n_model, n_bmodel):    
    
    if type(n_model)==str:
        models = []
        fnm = [f'{i}-grams.pkl' for i in [1, 2, 3]]
        for f in fnm:
            with open(f"{n_model}/{f}", "rb") as fp:
                models.append(pickle.load(fp))
        n_model = models
        
    len_tseed = []
    len_tseed_rev = []
    
    score = {k.order:[0, 0, 0] for k in n_model}
        
    for i, (in_, ans_) in enumerate(zip(inp, crct_ans)):
        sear = re.search(r"\*", in_)
        in_r = in_[::-1]
        sear_rev = re.search(r"\*", in_r)
        if sear:
            ix = sear.start()
            ix_rev = sear_rev.start()
        else:
            ix = len(in_)
            ix_rev = len(in_)
            print("Fracko")
        
        t_se = in_[:ix].lower()
        t_se_rev = in_r[:ix_rev].lower()
        
        len_tseed.append(len(t_se.split()))
        len_tseed_rev.append(len(t_se_rev.split()))
        
        for model, bmodel in zip(n_model, n_bmodel):
            topk = generate_topkbi(model, bmodel, k=10, text_seed=word_tokenize(t_se), text_seed_rev=word_tokenize(t_se_rev))
            avgk = report_successk(ans_, topk)
            print(avgk)
            for idx, succ_k in enumerate(avgk):
                score[model.order][idx]+=succ_k 
            if any(avgk):
                print(in_, ans_)
                print(model.order)
                print(t_se)
                print(topk)
                print(avgk)
    return {k:[elm/len(inp) for elm in v] for k, v in score.items()}, len_tseed

In [40]:
def generate_topk(model, k, text_seed=None):
    text_seed = [] if text_seed is None else list(text_seed)
    # This is the base recursion case.
    context = (
        text_seed[-model.order + 1 :]
        if len(text_seed) >= model.order
        else text_seed
    )
    samples = model.context_counts(model.vocab.lookup(context))
    while context and not samples:
        context = context[1:] if len(context) > 1 else []
        samples = model.context_counts(model.vocab.lookup(context))
    t_ = sorted(samples.items(), key=lambda x: x[1], reverse=True)
#     print(t_[:k])
    samples = dict(t_)
    return list(samples.keys())[:k]

In [41]:
import pickle 
import os
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.tokenize import word_tokenize
from nltk.lm import MLE, Laplace, KneserNeyInterpolated


In [42]:
def generate_topkbi(model, bmodel, k=10, text_seed=None, text_seed_rev=None):
    text_seed = [] if text_seed is None else list(text_seed)
    text_seed_rev = [] if text_seed_rev is None else list(text_seed_rev)
    # This is the base recursion case.
    context = (text_seed[-model.order + 1 :] if len(text_seed) >= model.order else text_seed)
    context_rev = (text_seed_rev[-model.order + 1 :] if len(text_seed_rev) >= model.order else text_seed_rev)
    samples = model.context_counts(model.vocab.lookup(context))
    samples_rev = bmodel.context_counts(bmodel.vocab.lookup(context_rev))
    
    while context and not samples:
        context = context[1:] if len(context) > 1 else []
        samples = model.context_counts(model.vocab.lookup(context))
    
    while context_rev and not samples_rev:
        context_rev = context_rev[1:] if len(context_rev) > 1 else []
        samples_rev = bmodel.context_counts(bmodel.vocab.lookup(context_rev))

    samples.update(samples_rev)
    
    t_ = sorted(samples.items(), key=lambda x: x[1], reverse=True)
    samples = dict(t_)
    return list(samples.keys())[:k]

In [43]:
def experimentbiDir(name, data, data_name):
    print(f"Running experiment {name} with data {data_name}")
    if os.path.exists(name):
        if len(os.listdir(name))>1:
            print(f"Loading saved model on disk")
            models = []
            fnm = [f'{i}-grams.pkl' for i in [1, 2, 3]]
            for f in fnm:
                with open(f"{name}/{f}", "rb") as fp:
                    models.append(pickle.load(fp))
            n_model = models
        if len(os.listdir(f'{name}_rev'))>1:
            print(f"Loading saved model on disk")
            models = []
            fnm = [f'{i}-grams.pkl' for i in [1, 2, 3, 5, 10]]
            for f in fnm:
                with open(f"{name}_rev/{f}", "rb") as fp:
                    models.append(pickle.load(fp))
            n_bmodel = models
            avgk, len_tseed = evaluate_spell_bi(inp, crct_ans, n_model, n_bmodel)
    else:
        n_model = []
        n_bmodel = []
        
        data_prep = data_prepfn(data)
        data_rev = [elem[::-1] for elem in data]
        data_prep_rev = data_prepfn(data_rev)
        print(f"Training and saving Frwd model on disk")
        n_model = dump_modelK(name, data_prep)
        print(f"Training and saving Reverse model on disk")
        n_bmodel = dump_modelK(f'{name}_rev', data_prep_rev)
        avgk, len_tseed = evaluate_spell_bi(inp, crct_ans, n_model, n_bmodel)
    print(avgk)
    return avgk, len_tseed
    

In [44]:
import json, pytrec_eval
def report_successk(crctspell, topkspell):
    qrel = {
        'q1': {
            f'{crctspell}': 1
        }
    }

    run = {
        'q1': {k:v for k,v in zip(topkspell, list(range(-1,-(len(topkspell)+1), -1)))}
    }
#     print(f"{qrel}\n{run}")
    evaluator = pytrec_eval.RelevanceEvaluator(qrel, {'success_1', 'success_5', 'success_10'})
    eval_ = evaluator.evaluate(run)['q1']
#     print(eval_)
    return eval_['success_1'], eval_['success_5'], eval_['success_10']


In [45]:
def experiment(name, data, data_name):
    print(f"Running experiment {name} with data {data_name}")
    if os.path.exists(name):
        if len(os.listdir(name))>1:
            print(f"Loading saved model on disk")
            avgk, len_tseed = evaluate_spell(inp, crct_ans, name)
    else:
        n_model = []
        
        data_prep = data_prepfn(data)
        print(f"Training and saving model on disk")
        n_model = dump_modelK(name, data_prep)
        avgk, len_tseed = evaluate_spell(inp, crct_ans, n_model)
    print(avgk)
    return avgk, len_tseed
    

In [25]:
import numpy as np


data = brown.sents(categories="news")
avgk, len_tseed = experiment("brown-newsK", data, "brown-news")

In [94]:
## brown news       
{1: 0.0, 2: 0.0101010101010101, 3: 0.026936026936026935, 5: 0.026936026936026935, 10: 0.026936026936026935}

{1: 0.0,
 2: 0.0101010101010101,
 3: 0.026936026936026935,
 5: 0.026936026936026935,
 10: 0.026936026936026935}

In [ ]:
##"belles_lettres", "learned"
# {1: 0.0, 2: 0.011784511784511785, 3: 0.021885521885521883, 5: 0.021885521885521883, 10: 0.021885521885521883}

In [ ]:
# brown-all
{1: 0.0, 2: 0.010101010101010102, 3: 0.02693602693602694}

In [ ]:
# brown-news Lap
{1: 0.0, 2: 0.0101010101010101, 3: 0.026936026936026935, 5: 0.026936026936026935, 10: 0.026936026936026935}
# brow-news Key
{1: 0.0, 2: 0.0101010101010101, 3: 0.026936026936026935, 5: 0.026936026936026935, 10: 0.026936026936026935}

## Running distil bert for the same task

In [6]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='distilbert-base-uncased')
resp = unmasker("Hello I'm a [MASK] model.")
topk = [elm['token_str']for elm in resp]
topk

['role', 'fashion', 'business', 'model', 'modeling']

In [30]:
sum_ = [0, 0, 0]
for i, (in_, ans_) in enumerate(zip(inp, crct_ans)):
    new_inp = re.sub("\*", '[MASK]', in_)
    resp = unmasker(new_inp)
    topk = [elm['token_str']for elm in resp]
    avgk = report_successk(ans_, topk)
    print(avgk)
    for idx, s_k in enumerate(avgk):
        sum_[idx]+=s_k
    if any(avgk):
        print(in_, ans_)
        print(topk)
        print(avgk)
succ_k = {k:v/len(inp) for k, v in zip([1, 5, 10], sum_)}
print(succ_k)

(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(1.0, 1.0, 1.0)
in the * when it was snowing winter
['winter', 'summer', 'morning', 'mornings', 'afternoon']
(1.0, 1.0, 1.0)
(0.0, 0.0, 0.0)
(0.0, 1.0, 1.0)
everything * the houses except
['in', 'except', 'around', 'inside', 'but']
(0.0, 1.0, 1.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 1.0, 1.0)
the wind * the leaves blew
['blows', 'carries', 'shakes', 'blew', 'rustling']
(0.0, 1.0, 1.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 1.0, 1.0)
making any * noise
['difference', '.', 'noise', 'progress', 'decisions']
(0.0, 1.0, 1.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(0.0, 0.0, 0.0)
(1.0, 1.0, 1.0)
an * in his hand arrow
['arrow', 'envelope', 'apple', 'axe', 'object']
(1.0, 1.0, 1.0)
(0.0, 1.0, 1.0)
they * a hole dig
['dug', 'dig', 'found', 'cut', '

In [ ]:
#transformer
# {1: 0.06565656565656566, 5: 0.20707070707070707, 10: 0.20707070707070707}